In [46]:
%pip install earthengine-api -q
%pip install folium -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import ee

In [2]:
ee.Authenticate()


Successfully saved authorization token.


In [3]:
ee.Initialize()

In [36]:
collection = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CO').select('CO_column_number_density').filterDate('2019-06-01', '2019-06-11')

In [35]:
import folium

def add_ee_layer(self, ee_image_object, vis_params, name, opacity = 0.5):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True,
        opacity=opacity,
    ).add_to(self)
    
folium.Map.add_ee_layer = add_ee_layer

In [23]:
import pandas as pd

def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

In [34]:
def get_ee_data(collection: ee.Collection, position: tuple[float, float], scale: int, bands: list[str]) -> pd.DataFrame:
    point = ee.Geometry.Point(*position)
    data = collection.getRegion(point, scale).getInfo()
    return ee_array_to_df(data, bands)

In [32]:
get_ee_data(collection, (4.8148, 45.7758), 1000, ["CO_column_number_density"])

,time,datetime,CO_column_number_density
6,1559389744000,2019-06-01 11:49:04,0.031941
20,1559475003000,2019-06-02 11:30:03,0.031068
34,1559560263000,2019-06-03 11:11:03,0.037147
49,1559651613000,2019-06-04 12:33:33,0.032898
63,1559736871000,2019-06-05 12:14:31,0.028632
77,1559822131000,2019-06-06 11:55:31,0.035523
91,1559907389000,2019-06-07 11:36:29,0.034918
105,1559992649000,2019-06-08 11:17:29,0.031781
106,1559998739000,2019-06-08 12:58:59,0.031931


In [40]:
def display_ee_data(collection: ee.Collection, position: tuple[float, float], opacity: float = 0.5):
    band_viz = {
        "min": 0,
        "max": 0.05,
        "palette": ["black", "blue", "purple", "cyan", "green", "yellow", "red"],
    }
    my_map = folium.Map(location=[position[0], position[1]], zoom_start=10)
    my_map = my_map.add_ee_layer(collection.mean(), band_viz, "S5P CO", opacity=opacity)
    
    return my_map


In [42]:
M = display_ee_data(collection, (4.8148, 45.7758), 0.5)